In [36]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from pathlib import Path
from tqdm import tqdm_notebook # or tqdm
import pandas as pd
import numpy as np
import subprocess as sp
import time
import json

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
!cd _data/_test_matching && ls

1068 - Atrévete Te Te.mp3      86564 - Persiana Americana.mp4
1106 - Perdóname.mp3	       86775 - Que Levante La Mano.mp4
787 - Rolling In The Deep.mp3  88202 - Ya Lo Pasado Pasado.mp4


In [22]:
root_path = Path('_data/_test_matching')

In [23]:
media_extensions = ['.avi', '.cdg', '.dat', '.mkv', '.mp3', '.mp4', '.mpg', '.wav', '.wma', '.wmv']
song_paths = []
file_paths = []
for song_file in root_path.iterdir():
    start = time.time()
    song_paths.append(song_file)
    extension = song_file.suffix
    if extension in media_extensions:
        sp.call(['essentia_streaming_extractor_music',
                 song_file, str(song_file).replace(extension, '.json')])
        file_paths.append(Path(str(song_file).replace(extension, '.json')))
        print(song_file, f'{time.time() - start:.2f}')

_data/_test_matching/1068 - Atrévete Te Te.mp3 13.09
_data/_test_matching/1106 - Perdóname.mp3 12.65
_data/_test_matching/787 - Rolling In The Deep.mp3 19.23
_data/_test_matching/86564 - Persiana Americana.mp4 17.74
_data/_test_matching/86775 - Que Levante La Mano.mp4 10.28
_data/_test_matching/88202 - Ya Lo Pasado Pasado.mp4 13.53


In [26]:
from files_diff import preprocess_dataframe, get_dataframe

In [30]:
songs_df = preprocess_dataframe(get_dataframe(song_paths))

In [32]:
songs_df['descriptor_path'] = file_paths

In [33]:
songs_df

,path,name,extension,code,descriptor_path
0,_data/_test_matching/1068 - Atrévete Te Te.mp3,1068 - atrévete te te,.mp3,NaN,_data/_test_matching/1068 - Atrévete Te Te.json
1,_data/_test_matching/1106 - Perdóname.mp3,1106 - perdóname,.mp3,NaN,_data/_test_matching/1106 - Perdóname.json
2,_data/_test_matching/787 - Rolling In The Deep...,787 - rolling in the deep,.mp3,NaN,_data/_test_matching/787 - Rolling In The Deep...
3,_data/_test_matching/86564 - Persiana American...,86564 - persiana americana,.mp4,NaN,_data/_test_matching/86564 - Persiana American...
4,_data/_test_matching/86775 - Que Levante La Ma...,86775 - que levante la mano,.mp4,NaN,_data/_test_matching/86775 - Que Levante La Ma...
5,_data/_test_matching/88202 - Ya Lo Pasado Pasa...,88202 - ya lo pasado pasado,.mp4,NaN,_data/_test_matching/88202 - Ya Lo Pasado Pasa...


In [38]:
for idx, row in songs_df.iterrows():
    features = json.load(open(row['descriptor_path']))
    songs_df.loc[idx, 'length'] = features['metadata']['audio_properties']['length']
    songs_df.loc[idx, 'bpm'] = features['rhythm']['bpm']
    songs_df.loc[idx, 'chords_key'] = features['tonal']['chords_key']
    songs_df.loc[idx, 'chords_scale'] = features['tonal']['chords_scale']

In [39]:
songs_df

,path,name,extension,code,descriptor_path,length,bpm,chords_key,chords_scale
0,_data/_test_matching/1068 - Atrévete Te Te.mp3,1068 - atrévete te te,.mp3,NaN,_data/_test_matching/1068 - Atrévete Te Te.json,252.970856,89.786362,D,minor
1,_data/_test_matching/1106 - Perdóname.mp3,1106 - perdóname,.mp3,NaN,_data/_test_matching/1106 - Perdóname.json,240.040253,133.140121,D,major
2,_data/_test_matching/787 - Rolling In The Deep...,787 - rolling in the deep,.mp3,NaN,_data/_test_matching/787 - Rolling In The Deep...,245.029633,105.048813,F,minor
3,_data/_test_matching/86564 - Persiana American...,86564 - persiana americana,.mp4,NaN,_data/_test_matching/86564 - Persiana American...,279.823669,100.923264,G,major
4,_data/_test_matching/86775 - Que Levante La Ma...,86775 - que levante la mano,.mp4,NaN,_data/_test_matching/86775 - Que Levante La Ma...,192.446991,90.953445,A#,major
5,_data/_test_matching/88202 - Ya Lo Pasado Pasa...,88202 - ya lo pasado pasado,.mp4,NaN,_data/_test_matching/88202 - Ya Lo Pasado Pasa...,251.036728,76.272369,C,major
